In [1]:
import numpy as np
import copy
from time import perf_counter
from collections import Counter, deque
from functools import lru_cache
import numba
from itertools import permutations, combinations, product

In [2]:
f = open("input.txt", "r")
rawstring = f.read()
f.close()

In [3]:
class Dice():
    def __init__(self) -> None:
        self.state = 1
        self.count = 0
    
    def roll(self):
        self.count += 1
        retVal = self.state
        self.incState()
        return retVal

    def incState(self, n=1):
        self.state += n
        self.state = self.state % 101
        if not( 0 < self.state <= 100):
            self.incState()

In [4]:
class Player():
    def __init__(self, startPos,dice=None) -> None:
        self.Pos = startPos
        self.score = 0
        self.won = False
        self.dice = dice
    
    def advance(self):
        self.Pos +=  self.dice.roll() + self.dice.roll() + self.dice.roll()
        while self.Pos > 10:
            self.Pos -= 10
        self.score += self.Pos

In [5]:
dice = Dice()
p1 = Player(10, dice)
p2 = Player(2, dice)

In [6]:
while(True):
    p1.advance()
    if p1.score >= 21:
        break
    p2.advance()
    if p2.score >= 21:
        break

part 2:

In [9]:
outcomes,weights = np.unique(np.array(list(
    product([1,2,3],[1,2,3],[1,2,3])
    )).sum(axis=1), return_counts=True)

outcomes = [outcomes[i].item() for i in range(len(outcomes))]
weights = [weights[i].item() for i in range(len(weights))]


In [10]:
type(outcomes[0])

int

In [11]:
type(outcomes[0])

int

In [12]:
type(outcomes[0])

int

rolling the dice three times can increase the players position by 3-9 marks.

In [19]:
mapping = dict.fromkeys(list(product(range(11),range(1001))))

for place, score in mapping.keys():
    temp = {}
    for outcome,i in zip(outcomes, weights):
        newPlace = place + outcome
        while newPlace > 10:
            newPlace -= 10
        newScore = score + newPlace
        temp[(newPlace, newScore)] = i
    mapping[(place, score)] = temp


In [21]:
def filterWon(inputdict):
    retdict = dict()
    count = 0
    for k,v in inputdict.items():
        if k[1] >= 1000:
            count += v
        else:
            retdict[k] = v
    return retdict,count

In [26]:

won = []

def perform_step(states,prev=None,iter=0):
    state = [
        {k:(v_inner*v) for k,v_inner in mapping[(k[0],k[1])].items()}
        for k,v in states.items()]
    new_dict = dict()
    # consoliding the list of dicts into a dict
    for sub_dict in state:
        for k in sub_dict.keys():
            if k in new_dict:
                new_dict[k] += sub_dict[k]
            else:
                new_dict[k] = sub_dict[k]

    # filtering out everything that has won already
    new_dict,c = filterWon(new_dict)
    won.append(c)
    return new_dict
    
input_1 = {(4,0): 1}
output_1 = perform_step(input_1)
input_1 = output_1
input_2 = {(8,0): sum(output_1.values())}
output_2 = perform_step(input_2)
input_2 = output_2

i = 0
while True:
    i += 1

    # normalizing the input_1 so player_1 begins his turn in as many universes
    # as player 2 finished his last turn.
    input_1 = {k:(v*
    # this factor is 3**3 = 27, unless the previous player won some games
    sum(output_2.values())//sum(input_1.values())
    ) for k,v in input_1.items()}

    output_1 = perform_step(input_1)
    input_1 = output_1
    if len(output_1) == 0:
        break
    input_2 = {k:(v*
        sum(output_1.values())//sum(input_2.values())
        ) for k,v in input_2.items()}
    output_2 = perform_step(input_2)
    input_2 = output_2
    if len(output_2) == 0:
        break


KeyboardInterrupt: 

In [24]:
sum(won[::2])

nan

In [25]:
sum(won[1::2])

nan